In [1]:
import os
import string
import sys
import csv

In [ ]:
PROJECT_PATH = '/home/crypdick/Apps/masters/dbs/ds-homelessness-FL'
RAW_DATA_PATH = os.path.join(PROJECT_PATH, 'data', 'raw')
# MODEL_PATH = os.path.join(PROJECT_PATH, 'data', 'model')
os.chdir(PROJECT_PATH)

# CSV_fnames = sorted(os.listdir(os.path.join(os.path.realpath('.'), RAW_DATA_PATH)))
CSV_fnames = sorted([fname for fname in os.listdir(RAW_DATA_PATH) if fname.endswith('.csv')])
CSV_full_dirs = [os.path.join(RAW_DATA_PATH, fname) for fname in CSV_fnames]

# print(CSV_fnames)
# print(CSV_full_dirs)

temp_list = []
temp_list.append(os.path.join(RAW_DATA_PATH, 'temp.csv'))



test

In [57]:
MIN_ROW_LEN = 35
N_LEADING_COLS = 8  # all entries have this many leading cols
N_TRAILING_COLS = -14  #13
for csv_file in temp_list:
    with open(csv_file, 'r', newline='') as open_file:
        with open('joined_list.csv','w') as joined_csv:
            reader = csv.reader(open_file, delimiter=',', quotechar='"')
            writer = csv.writer(joined_csv)
            joined_row = []
            extra_crimes = []
            for i, row in enumerate(reader):
                if 'RELEASE DATE' in row[0]:
                    joined_row.extend(row)
                    name = joined_row[0]
                    booking_n = joined_row[1]
                    agency = joined_row[4]
                    ABN = joined_row[5]
                    race_sex_eth_DOB = joined_row[7]
                    # race = race_sex_eth_DOB.popleft()
                    race, sex, ethnicity, DOB = race_sex_eth_DOB.split(" / ")
                    crimes_vec = joined_row[N_LEADING_COLS:N_TRAILING_COLS]
                    crimes_list = []
                    while len(crimes_vec) >= 1:
                        print("loop start vec", crimes_vec)
                        single_crime = crimes_vec[:4]
                        if single_crime[0].strip() != '':
                            print("storing crime: ", single_crime)
                            crimes_list.append(single_crime)
                        garbage = crimes_vec[4:7]
                        #print("trash", garbage)
                        crimes_vec = crimes_vec[7:]
                    if len(crimes_vec) ==  13:  # single crime == len 13
                        pass
                        #print(crimes_vec)
                        #print("chargeType {},charge {},court {},caseNumber {}".format(crimes_vec[0], crimes_vec[1], crimes_vec[2], crimes_vec[3]))
                    else:
                        #print(crimes_vec[4:])
                        #print("chargeType {},charge {},court {},caseNumber {}".format(crimes_vec[0], crimes_vec[1], crimes_vec[2], crimes_vec[3]))
                        extra_crimes = False
                    
                    # TODO: split DOB
                    #print(joined_row[:N_LEADING_COLS])
                    #print("name {}, booking# {}, agency {}, ABN {}, race sex ethnicity dob {}".format(joined_row[0], joined_row[1], joined_row[4], joined_row[5], joined_row[7]))
                    #print(len(crimes_vec), crimes_vec)
                    trailing_vec = joined_row[N_TRAILING_COLS:]
                    address = "{},{}".format(trailing_vec[0], trailing_vec[1])
                    address = address[9:]   # strip "ADDRESS: "
                    place_of_birth = trailing_vec[2]
                    place_of_birth = place_of_birth[5:]  # strip "POB: "
                    releaseDate = trailing_vec[7]
                    releaseDate = releaseDate[14:]  # strip "RELEASE DATE: "
                    releaseCode = trailing_vec[8]
                    releaseCode = releaseCode[14:]    # strip "RELEASE CODE: "
                    SOID = trailing_vec[9]
                    SOID = SOID[6:]    # strip "SOID: "
                    #print(address,place_of_birth,releaseDate,releaseCode,SOID)
                    #clean_booking_vec = [name, booking_n, agency, ABN, race, sex, ethnicity, DOB, chargeType,charge,court,caseNumber,address,POB,releaseDate,releaseCode,SOID]
                    if len(joined_row) == MIN_ROW_LEN:
                        joined_row = []
                    elif len(joined_row) > MIN_ROW_LEN:
                        #print(len(joined_row))
                        
                        joined_row = []
                        extra_crimes = []
                    else:
                        raise Exception("Not expecting rows with less than {} columns".format(MIN_ROW_LEN))
                    # writer.writerow(joined_row)
                    
                    
                else:
                    joined_row.extend(row)
                    
                
            # if release, append, write row, then clear true and release
            # else, append, dont write
            
            # print(', '.join(row))
            # clean: name 0, booking# 1, agency 4, ABN 5, race sex ethnicity dob 7,   
            
            
            

loop start vec ['CAPIAS ', 'CONSUMING ON STREETS ', '41C ', '9933579MO', '', '', '  ', 'CAPIAS ', 'POSSESSION OF OPEN CONTAINER ', '41C ', '9922942MO', '', '', '  ', ' ', '', '', '', '', '']
storing crime:  ['CAPIAS ', 'CONSUMING ON STREETS ', '41C ', '9933579MO']
loop start vec ['CAPIAS ', 'POSSESSION OF OPEN CONTAINER ', '41C ', '9922942MO', '', '', '  ', ' ', '', '', '', '', '']
storing crime:  ['CAPIAS ', 'POSSESSION OF OPEN CONTAINER ', '41C ', '9922942MO']
loop start vec [' ', '', '', '', '', '']
loop start vec ['CAPIAS ', 'OBTAIN.PROP. FOR WORTHLESS CK.(LESS THAN $ ', '99', 'CTC9820905MMANO', '', '', '  ', ' ', '', '', '', '', '']
storing crime:  ['CAPIAS ', 'OBTAIN.PROP. FOR WORTHLESS CK.(LESS THAN $ ', '99', 'CTC9820905MMANO']
loop start vec [' ', '', '', '', '', '']
loop start vec ['CAPIAS ', 'GRAND THEFT THIRD DEGREE ($300 - $20,000) ', '31O ', '9912353', '', '', '  ', 'CAPIAS ', 'OBSTRUCTING OR OPPOSING AN OFFICER WITHOUT ', '31O ', '9912353', '', '', '  ', 'CAPIAS ', 'TRES